In [ ]:
import re
def filter_special_character(docs):
    # remove ., needed for bert
    filter_patter = r'''[\w']+|[!?.,;-~{}`´_<=>:/@*()&'$%#"]'''
    return [re.findall(filter_patter, docs[doc_idx]) for doc_idx in range(len(docs))]

docs = [
        'anbv 8523> $, asfnoa 53§$%3 hallo. i am',
        'halloe 1& §&!, name' , 'am'
]
raw_docs = [" ".join(doc) for doc in filter_special_character(docs)]
raw_docs

['anbv 8523 > $ , asfnoa 53 $ % 3 hallo . i am', 'halloe 1 & & ! , name', 'am']

In [55]:
from sklearn.datasets import fetch_20newsgroups
def read_raw_documents():
    """
    newsgroups_train = fetch_20newsgroups(subset='train')
    raw_documents = []
    raw_labels = []
    for i in range(0,len(newsgroups_train.data)):
        raw_documents.append(newsgroups_train.data[i])
        raw_labels.append(newsgroups_train.target[i])
    """
    train_data = fetch_20newsgroups(subset='train')
    test_data = fetch_20newsgroups(subset='test')
    def filter_special_character(docs):
        # remove ., needed for bert
        filter_patter = r'''[\w']+|[!?.,;-~{}`´_<=>:/@*()&'$%#"]'''
        return [re.findall(filter_patter, docs[doc_idx]) for doc_idx in range(len(docs))]
    init_docs_tr = filter_special_character(train_data.data)
    init_docs_ts = filter_special_character(test_data.data)
    complete_documents = init_docs_tr + init_docs_ts
    raw_documents = []
    for doc in complete_documents:
        raw_documents.append(" ".join(doc))
    return raw_documents, None
raw_documents,_ = read_raw_documents()

In [59]:
train_data = fetch_20newsgroups(subset='train').data
test_data = fetch_20newsgroups(subset='test').data
original_docs = train_data + test_data
print(original_docs[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [60]:
raw_documents[0]

"From : lerxst @ wam . umd . edu ( where's my thing ) Subject : WHAT car is this ! ? Nntp Posting Host : rac3 . wam . umd . edu Organization : University of Maryland , College Park Lines : 15 I was wondering if anyone out there could enlighten me on this car I saw the other day . It was a 2 door sports car , looked to be from the late 60s / early 70s . It was called a Bricklin . The doors were really small . In addition , the front bumper was separate from the rest of the body . This is all I know . If anyone can tellme a model name , engine specs , years of production , where this car is made , history , or whatever info you have on this funky looking car , please e mail . Thanks , IL brought to you by your neighborhood Lerxst"

In [66]:
 str, l = clean_doc_for_bert(raw_documents[0])
 print(str)

from lerxst wam . umd . edu where my thing subject what car is this nntp posting host . wam . umd . edu organization university of maryland , college park lines was wondering if anyone out there could enlighten me on this car saw the other day . it was door sports car , looked to be from the late early . it was called bricklin . the doors were really small . in addition , the front bumper was separate from the rest of the body . this is all know . if anyone can tellme model name , engine specs , years of production , where this car is made , history , or whatever info you have on this funky looking car , please mail . thanks , il brought to you by your neighborhood lerxst


In [65]:
def contains_numeric(w):
        return any(char.isdigit() for char in w)
    
def only_letters(tested_string):
    for letter in tested_string:
        if letter not in "abcdefghijklmnopqrstuvwxyz":
            return False
    return True

def clean_doc_for_bert(doc): 
    #word_list = doc.split(" ") #doc.replace(">","").lower()
    word_list = word_tokenize(doc.lower()) #only using empty space and punctation for tokenization
    cleaned = []
    for w in word_list:
        #if w not in stop_words:
          #if w in string.punctuation or only_letters(w): #using only character from punctation and alpha characters
          if not contains_numeric(w):
              if w in ['.', ','] or only_letters(w):
                  if w in string.punctuation or len( set(w) ) > 1: #punctation with len 1 allowed but alpha word must be longer then 1
                      cleaned.append( w)
    return " ".join(cleaned), cleaned  #save doc in string and in token-list         

In [62]:
from nltk.tokenize import word_tokenize
import nltk
import string
nltk.download('punkt')

def simple_preprocess(raw_documents):
    cleaned_documents = []
    for doc in raw_documents:
        doc_in_string, doc_in_token_list = clean_doc_for_bert(doc)
        cleaned_documents.append(doc_in_string)
    return cleaned_documents

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
a = simple_preprocess(raw_docs)
print(a)

['anbv , asfnoa hallo . am', 'halloe , name', 'am']


In [67]:
def transform_to_sentences(docs): #no labels
    data_as_sentences = []
    for doc in docs:
      for sent in doc.split("."): #make sentences
        #print(f'after split: {sent}')
        updated_sent = " ".join([t for t in sent.strip().split(" ") if len(t) > 1])
        #print(f'update: {updated_sent}')
        if len(updated_sent.split(" ")) > 1:
            data_as_sentences.append(updated_sent)
        else:
            if updated_sent not in data_as_sentences:
                data_as_sentences.append(updated_sent)
    return data_as_sentences

In [70]:
pr_docs = simple_preprocess(raw_documents[:2])
for d in pr_docs:
  print(d)
print('-'*100)
sents = transform_to_sentences(pr_docs)
for s in sents:
  print(s)

from lerxst wam . umd . edu where my thing subject what car is this nntp posting host . wam . umd . edu organization university of maryland , college park lines was wondering if anyone out there could enlighten me on this car saw the other day . it was door sports car , looked to be from the late early . it was called bricklin . the doors were really small . in addition , the front bumper was separate from the rest of the body . this is all know . if anyone can tellme model name , engine specs , years of production , where this car is made , history , or whatever info you have on this funky looking car , please mail . thanks , il brought to you by your neighborhood lerxst
from guykuo carson . . washington . edu guy kuo subject si clock poll final call summary final call for si clock reports keywords si , acceleration , clock , upgrade article . . shelley . organization university of washington lines nntp posting host carson . . washington . edu fair number of brave souls who upgraded t

diese Sätze sind anders von dem bert_sentences.txt wegen der stopwords noch nicht gefiltern sind